In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('train.csv',encoding_errors= 'replace')

In [3]:
data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km�),Density (P/Km�)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [4]:
data = data.drop(columns= ['textID','text','Time of Tweet','Age of User','Country','Population -2020','Land Area (Km�)','Density (P/Km�)'])

In [5]:
data = data.rename(columns = {'selected_text':'text'})

In [6]:
data.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
encoder = LabelEncoder()

In [9]:
data['sentiment'] = encoder.fit_transform(data['sentiment'])

In [10]:
data.head()

,text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD,0
2,bullying me,0
3,leave me alone,0
4,"Sons of ****,",0


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [12]:
import nltk
import nltk

# Download stopwords
nltk.download('stopwords')

# Download wordnet for lemmatization
nltk.download('wordnet')

from nltk.corpus import stopwords
import string

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [13]:
def clean_data(mess):
    nopunc=[i for i in mess if i not in string.punctuation]
    nopunc= ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [14]:
text = 'me anf you going to eat tomorrowm'

nopunc=[i for i in text if i not in string.punctuation]

nopunc= ''.join(nopunc)

end_text =[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [15]:
import re

def clean_tweet(tweet):
    if isinstance(tweet, str):  # Check if tweet is a string
        tweet = tweet.lower()  # Lowercase the text
        tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)  # Remove URLs
        tweet = re.sub(r'\@\w+|\#', '', tweet)  # Remove mentions and hashtags
        # Add any other cleaning steps you want here
    else:
        tweet = ''  # You can set it to an empty string or handle it as needed
    return tweet


In [16]:
data['text'] = data['text'].apply(clean_tweet)

In [17]:
data.head()

,text,sentiment
0,"i`d have responded, if i were going",1
1,sooo sad,0
2,bullying me,0
3,leave me alone,0
4,"sons of ****,",0


In [18]:
X = data['text']
y = data['sentiment']

In [19]:
data.head()

,text,sentiment
0,"i`d have responded, if i were going",1
1,sooo sad,0
2,bullying me,0
3,leave me alone,0
4,"sons of ****,",0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
pipe = Pipeline(
    [
        ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=10000)),
        ('svm', SVC(kernel='linear', random_state=42))
    ]
)

In [22]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=10000, ngram_range=(1, 2))),
                ('svm', SVC(kernel='linear', random_state=42))])

In [23]:
'''from sklearn.model_selection import GridSearchCV
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__class_weight': ['balanced', None]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Evaluate the best model
best_model = grid_search.best_estimator_
'''

"from sklearn.model_selection import GridSearchCV\nparam_grid = {\n    'classifier__n_estimators': [100, 200],\n    'classifier__max_depth': [None, 10, 20],\n    'classifier__min_samples_split': [2, 5],\n    'classifier__class_weight': ['balanced', None]\n}\n\ngrid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)\ngrid_search.fit(X_train, y_train)\n\n# Evaluate the best model\nbest_model = grid_search.best_estimator_\n"

In [24]:
prediction = pipe.predict(X_test)

In [25]:
prediction.max()

2

In [26]:
from sklearn.metrics import classification_report,accuracy_score

In [27]:

print("Accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test,  prediction, target_names=encoder.classes_))

Accuracy: 0.8242571255306246
              precision    recall  f1-score   support

    negative       0.79      0.78      0.79      2338
     neutral       0.80      0.88      0.84      3371
    positive       0.90      0.79      0.84      2536

    accuracy                           0.82      8245
   macro avg       0.83      0.82      0.82      8245
weighted avg       0.83      0.82      0.82      8245



In [35]:
text= [
    "im sorry to hear about your dad"
]

In [36]:
import joblib
joblib.dump(pipe,'sentiment_model.pkl')


['sentiment_model.pkl']

In [37]:
# Load the model from the file
pipeline = joblib.load('sentiment_model.pkl')

In [38]:
vectorizer= TfidfVectorizer()

In [39]:
pred = pipe.predict(text)

In [40]:
pred

array([0])

In [41]:
# Output predictions
for test, sentiment in zip(text, pred):
    if sentiment == 0:
        print('Negative')
    elif sentiment == 1:
        print('neutral')
    else:
        print('positive')

Negative
